# COPD Hospitalisations | Processing

## Set Up

Ensure that the required libraries are available by running the below code in the terminal before execution:
- pip install pandas


Execute the following in the jupyter notebook before execution to ensure that the required libraries are imported:

In [101]:
import pandas as pd

## Load Dataset

In [102]:
df = pd.read_csv('raw.csv')

## Data Manipulation

In [103]:
# Rename columns to match Air Quality data set.
df = df.rename(columns={'LHD': 'lhd', 'Period': 'financial year'})                                                            # Rename columns
df.columns = df.columns.str.lower()                                                                                 # Lowercase column names

# Remove ' LHD' for Local Health District values.
df['lhd'] = df['lhd'].str.replace(' LHD', '')                                                                       # Remove ' LHD' from lhd column

# Remove rows representing state-wide aggregated data.
df = df.dropna(subset=['lhd'])                                                                                      # Remove rows with missing values in 'lhd' column
df = df[~df['lhd'].str.contains('All')]                                                                             # Remove rows with 'All' in 'lhd' column

# Remove columns holding Confidence Interval data.
df = df.loc[:, ~df.columns.str.contains('% ci')]                                                                    # Remove columns with '% ci' in the name

# Reformat 'financial year' values from XX/YY to XXXX/YYYY.
df['financial year'] = df['financial year'].apply(                                                                  # Add '20' to the start of each year value
    lambda x: f'20{x[:2]}/20{x[3:]}' if isinstance(x, str) else x
)

# Remove rows where 'risk group' is not 'All ages'.
df = df[df['risk group'] == 'All ages']                                                                             # Keep rows where 'risk group' is 'All ages'
df = df.drop(columns=['risk group'])                                                                                # Drop 'risk group' column

# View the cleaned data.
df.head()                                                                                                           # Display the first few rows of the cleaned data

,sex,lhd,financial year,"rate per 100,000 population"
2020,Males,Sydney,2001/2002,313.6
2021,Males,Sydney,2002/2003,282.3
2022,Males,Sydney,2003/2004,298.8
2023,Males,Sydney,2004/2005,326.7
2024,Males,Sydney,2005/2006,283.7


In [105]:
# Convert 'rate per 100,000 population' to numeric, coercing errors to NaN
df['rate per 100,000 population'] = pd.to_numeric(df['rate per 100,000 population'], errors='coerce')

# Pivot the dataframe to have 'sex' as columns
df = df.pivot_table(index=['financial year', 'lhd'], columns='sex', values='rate per 100,000 population').reset_index()

# Rename the columns to match the desired format
df.columns.name = None
df = df.rename(columns={
    'Persons': 'Persons rate per 100,000 population',
    'Males': 'Male rate per 100,000 population', 
    'Females': 'Female rate per 100,000 population'
})

# View the DataFrame
df.head() 

,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
0,2001/2002,Central Coast,235.4,348.2,281.1
1,2001/2002,Far West,241.6,362.2,289.3
2,2001/2002,Hunter New England,210.8,319.9,257.9
3,2001/2002,Illawarra Shoalhaven,190.9,301.5,239.7
4,2001/2002,Mid North Coast,268.1,351.3,303.1


## Set Date Range

Set the range of financial years from 2014/2015 to 2023/2024.

In [106]:
# Drop pre 2014/2015 data.
df = df[~df['financial year'].isin(['2011/2012', '2012/2013', '2013/2014'])]                                        # Drop pre 2014/2015 data.

# Add rows for each LHD for the missing years until 2023/2024.
missing_rows = []                                                                                                   # Create a list to store the missing rows.
lhds = df['lhd'].unique()                                                                                           # Get unique LHDs.
years = [f"{year}/{year + 1}" for year in range(2014, 2024)]                                                        # Create a list of years from 2014/2015 to 2023/2024.

for lhd in lhds:
    for year in years:
        if not ((df['lhd'] == lhd) & (df['financial year'] == year)).any():                                         # Check if the row is missing.
            missing_rows.append({                                                                                   # Append the missing row to the list.
                'lhd': lhd, 
                'financial year': year, 
                'Female rate per 100,000 population': None,
                'Male rate per 100,000 population': None,
                'Persons rate per 100,000 population': None,
            })   

# Create a DataFrame from the missing rows and concatenate it to the original DataFrame
if missing_rows:
    df_missing = pd.DataFrame(missing_rows)                                                                         # Create a DataFrame from the missing rows.
    df = pd.concat([df, df_missing], ignore_index=True)                                                             # Concatenate the DataFrames.

# Sort the DataFrame by 'lhd' and 'financial year'.
df = df.sort_values(by=['lhd', 'financial year']).reset_index(drop=True)                                           # Sort the DataFrame by 'lhd' and 'financial year'.

# View the DataFrame.
df.tail()                                                                                                          # View the last 5 rows of the DataFrame.

C:\Users\matth\AppData\Local\Temp\ipykernel_9728\3336287228.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_missing], ignore_index=True)                                                             # Concatenate the DataFrames.


,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
295,2019/2020,Western Sydney,193.5,204.9,197.4
296,2020/2021,Western Sydney,131.7,161.5,144.3
297,2021/2022,Western Sydney,118.9,127.3,121.3
298,2022/2023,Western Sydney,NaN,NaN,NaN
299,2023/2024,Western Sydney,NaN,NaN,NaN


Fill missing values using linear interpolation.

In [107]:
df['Female rate per 100,000 population'] = df['Female rate per 100,000 population'].interpolate()                                       # Fill missing values using linear interpolation.
df['Male rate per 100,000 population'] = df['Male rate per 100,000 population'].interpolate()                                        # Fill missing values using linear interpolation.
df['Persons rate per 100,000 population'] = df['Persons rate per 100,000 population'].interpolate()                                     # Fill missing values using linear interpolation.

## Output Processed Dataset

In [108]:
# Save the processed data to a new CSV file.
df.to_csv('processed.csv', index=False)                                                                            # Save the processed data to a new CSV file.

# View the DataFrame.
df.head()                                                                                                          # View the first 5 rows of the DataFrame.

,financial year,lhd,"Female rate per 100,000 population","Male rate per 100,000 population","Persons rate per 100,000 population"
0,2001/2002,Central Coast,235.4,348.2,281.1
1,2002/2003,Central Coast,228.1,334.0,270.9
2,2003/2004,Central Coast,219.0,377.1,283.0
3,2004/2005,Central Coast,251.2,390.6,308.7
4,2005/2006,Central Coast,237.8,368.5,292.8
